In [1]:
%load_ext autoreload
%autoreload 2
%reload_ext autoreload

In [1]:
import json
import os
import pandas as pd
from datetime import datetime
from dotenv import load_dotenv, find_dotenv
from pathlib import Path
from src.func import tweet_utils
from src.func import regex
from src.func import labmtgen
from src.sentiment.senti_utils import *
#from src.scripts.process_tweets import *
from labMTsimple.storyLab import *
import random
import numpy as np
from collections import Counter
from collections import defaultdict
import seaborn as sns
%matplotlib inline

In [2]:
# Loop through cities
tweet_dir= Path("../data/processed/park_user_tweets")
cities = list(tweet_dir.glob("*.json"))


# open stop words
stop_file = Path("../src/sentiment/city_stops.json")
with open(stop_file, 'r') as fp:
    stop_dict  = json.load(fp)

# Load a cities worth of user tweets
city = cities[10]
# with open(city, 'r') as f:
#    park_user_tweets = json.load(f)

In [3]:
def wordshift_parks(city, control_type='time'):
    
    # load tweets
    with open(city, 'r') as f:
        park_user_tweets = json.load(f)
    print(city.name, len(park_user_tweets)) 

    # get control text
    if control_type == "time":
        park_tweet_text, control_tweet_text, park_list = get_time_control_text(park_user_tweets)
    elif control_type == "user":
        park_tweet_text, control_tweet_text, park_list = get_user_control_text(park_user_tweets)
    # get stop words
    park_stops  = get_park_stopwords(park_list)
    stop_list = park_stops+stop_dict['all_cities']+ stop_dict[city.stem]
    stop_lower = [x.lower() for x in dict(regex.get_ngrams(' '.join(stop_list), path='../src/func/ngrams.bin')).keys()]
    
    ## get frequency vectors
    PVec = getVecs(park_tweet_text)
    CVec = getVecs(control_tweet_text)
    ## get stopped vectors and valence
    PVec, Pval = labmtSimpleSentiment(PVec,stop_lower)
    CVec, Cval = labmtSimpleSentiment(CVec,stop_lower)
    
    ## print resluts
    print(city.stem, ": Park: {:05.3f} | Control {:05.3f}".format(Pval, Cval))
    labMT,labMTvector,labMTwordList = emotionFileReader(stopval=1.0,lang='english',returnVector=True)
    
    ## do the shift and save
    shiftHtml(labMTvector,labMTwordList, CVec, PVec, "shifts/" + str(city.stem)+"_{}.html".format(control_type))
    
#for city in cities:
#    wordshift_parks(city, 'time')
#    wordshift_parks(city, 'user')

In [15]:
park_tweets_all = []
control_tweets_all = []
stop_list  = [j  for i in stop_dict.values() for j in i]

for city in tweet_dir.glob("*.json"):
    with open(city, 'r') as f:
        park_user_tweets = json.load(f)
    park_tweet_text, control_tweet_text, park_list = get_user_control_text(park_user_tweets)
    park_stops  = get_park_stopwords(park_list)
    stop_list += park_stops
    index_list = range(len(park_tweet_text))
    n = min(1000, len(park_tweet_text))
    sample = random.sample(index_list, n)
    park_tweet_sample = [park_tweet_text[i] for i in sample]
    control_tweet_sample = [control_tweet_text[i] for i in sample]
    print("City: {} w {} park users and {} samples".format(city, len(park_user_tweets),len(park_tweet_sample)))
    park_tweets_all += park_tweet_sample
    control_tweets_all += control_tweet_sample

City: ../data/processed/park_user_tweets/CO_Denver_0820000.json w 3902 park users
City: ../data/processed/park_user_tweets/CO_Denver_0820000.json w 3902 park users and 1000 samples
City: ../data/processed/park_user_tweets/AZ_Phoenix_0455000.json w 7566 park users
City: ../data/processed/park_user_tweets/AZ_Phoenix_0455000.json w 7566 park users and 1000 samples
City: ../data/processed/park_user_tweets/FL_Jacksonville_1235000.json w 3218 park users
City: ../data/processed/park_user_tweets/FL_Jacksonville_1235000.json w 3218 park users and 1000 samples
City: ../data/processed/park_user_tweets/TX_Austin_4805000.json w 14689 park users
City: ../data/processed/park_user_tweets/TX_Austin_4805000.json w 14689 park users and 1000 samples
City: ../data/processed/park_user_tweets/TX_Fort_Worth_4827000.json w 4278 park users
City: ../data/processed/park_user_tweets/TX_Fort_Worth_4827000.json w 4278 park users and 1000 samples
City: ../data/processed/park_user_tweets/TX_El_Paso_4824000.json w 1397

In [36]:
len(control_tweets_all)

26000

In [37]:
with open("../data/processed/combined_tweets/combined_tweet_sample_control_user.json", 'w') as fp:
    json.dump(control_tweets_all,fp)

In [33]:
all_stops = stop_lower
stop_dict['all_cities'].append('park')
new_stops = stop_dict['all_cities']
stop_lower = [x.lower() for x in dict(regex.get_ngrams(' '.join(new_stops), path='../src/func/ngrams.bin')).keys()]

In [35]:

## get frequency vectors
PVec = getVecs(park_tweets_all)
CVec = getVecs(control_tweets_all)
## get stopped vectors and valence
PVec, Pval = labmtSimpleSentiment(PVec,stop_lower)
CVec, Cval = labmtSimpleSentiment(CVec,stop_lower)

## print resluts
print(city.stem, ": Park: {:05.3f} | Control {:05.3f}".format(Pval, Cval))
labMT,labMTvector,labMTwordList = emotionFileReader(stopval=1.0,lang='english',returnVector=True)
shiftHtml(labMTvector,labMTwordList, CVec, PVec, "shifts/combined_city_user_lessstops.html")

IL_Chicago_1714000 : Park: 6.139 | Control 6.086
wrote shift to shifts/combined_city_user_lessstops.html


In [4]:
results_dir = Path("../data/processed/combined_tweets/tweet_samples_0421")
with open(results_dir/Path("park_tweets.json"), 'r') as fp:
    park_tweets = json.load(fp)
with open(results_dir/Path("time_tweets.json"), 'r') as fp:
    time_tweets = json.load(fp)
with open(results_dir/Path("user_tweets.json"), 'r') as fp:
    user_tweets = json.load(fp)
with open(results_dir/Path("park_stops.json"), 'r') as fp:
    park_stops = json.load(fp)

In [5]:
park_tweet_text = []
for city, tweets in park_tweets.items():
    park_tweet_text += tweets
time_tweet_text = []
for city, tweets in time_tweets.items():
    time_tweet_text += tweets
stops = []
for city, city_stops in park_stops.items():
    stops+=city_stops
user_tweet_text = []
for city, tweets in user_tweets.items():
    user_tweet_text += tweets

In [9]:
park_tweet_text[10300]

'Let the Black Friday madness begin'

In [10]:
stops = list(set([x.lower() for x in dict(regex.get_ngrams(' '.join(stops),
                path='../src/func/ngrams.bin')).keys()]))

freq_stops = []
for city, words in stop_dict.items():
    freq_stops+=words
freq_stops.append('park')
freq_stops = list(set(freq_stops))

all_stops = freq_stops + stops

from labMTsimple.storyLab import *
stops_w_val=[]
words = labmtgen.load_labmt_words("../data/raw/data_labmt_simple.txt")
senti_dict = words.set_index('Word').to_dict()['Happs']
for stop in all_stops:
    if stop in senti_dict.keys():
        val = senti_dict[stop]
        if val <= 4.0 or val >= 6.0:
            #count+=1
            stops_w_val.append(stop)
            #print(stop,senti_dict[stop])
# check how #many of stop words are in labmt outside 4-6 range
print(len(stops_w_val))

571


In [11]:
PVec = getVecs(park_tweet_text)

In [16]:
words[words.Word=='park']

,Word,Happs
518,park,7.08


In [23]:
## get frequency vectors
PVec = getVecs(time_tweet_text)
CVec = getVecs(user_tweet_text)
## get stopped vectors and valence
PVec, Pval = labmtSimpleSentiment(PVec,freq_stops)
CVec, Cval = labmtSimpleSentiment(CVec,freq_stops)

In [24]:
## print resluts
print("All cities"": Time control: {:05.3f} | User Control {:05.3f}".format(Pval, Cval))
labMT,labMTvector,labMTwordList = emotionFileReader(stopval=1.0,lang='english',returnVector=True)
shiftHtml(labMTvector,labMTwordList, CVec, PVec, "shifts/user_time_control_shift.html")

All cities: Time control: 6.046 | User Control 6.138
wrote shift to shifts/user_time_control_shift.html


In [3]:
results_dir = Path("../data/processed/combined_tweets/tweet_samples_0513/")
with open(results_dir/Path("park_tweets_bytype.json"), 'r') as fp:
    park_tweets_bytype = json.load(fp)


In [4]:
park_tweets_bytype.keys()

dict_keys(['acres_1_10', 'acres_gt_100', 'acres_10_100', 'acres_lt_1'])

In [7]:
len(park_tweets_bytype['acres_gt_100'])
len(park_tweets_bytype['acres_10_100'])

12500

In [17]:
## get frequency vectors
PVec = getVecs(park_tweets_bytype['acres_gt_100'])
CVec = getVecs(park_tweets_bytype['acres_10_100'])
## get stopped vectors and valence
PVec, Pval = labmtSimpleSentiment(PVec,all_stops)
CVec, Cval = labmtSimpleSentiment(CVec,all_stops)

In [18]:
print("All cities"": Park > 100: {:05.3f} | Park 10-100 {:05.3f}".format(Pval, Cval))
labMT,labMTvector,labMTwordList = emotionFileReader(stopval=1.0,lang='english',returnVector=True)
shiftHtml(labMTvector,labMTwordList, CVec, PVec, "shifts/park100_park10.html")

All cities: Park > 100: 5.939 | Park 10-100 5.897
wrote shift to shifts/park100_park10.html


In [26]:
results_dir = Path("../data/processed/time_tweets/time_tweets_0424/season/")
with open(results_dir/Path("park_tweets_season_summer.json"), 'r') as fp:
    summer_tweets = json.load(fp)
with open(results_dir/Path("park_tweets_season_winter.json"), 'r') as fp:
    winter_tweets = json.load(fp)

In [29]:
## get frequency vectors
PVec = getVecs(summer_tweets)
CVec = getVecs(winter_tweets)
## get stopped vectors and valence
PVec, Pval = labmtSimpleSentiment(PVec,all_stops)
CVec, Cval = labmtSimpleSentiment(CVec,all_stops)

In [30]:
print("Summer: {:05.3f} | Winter {:05.3f}".format(Pval, Cval))
labMT,labMTvector,labMTwordList = emotionFileReader(stopval=1.0,lang='english',returnVector=True)
shiftHtml(labMTvector,labMTwordList, CVec, PVec, "shifts/summer_winter_inparks.html")

Summer: 5.960 | Winter 5.884
wrote shift to shifts/summer_winter_inparks.html
